In [1]:
import os
import glob

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [3]:
import torch
import torchvision as tv
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms.functional as TF
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader

In [4]:
# CUDA stuff
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")


In [5]:
use_cuda

True

# use cycleGAN

## what to use?

for generator, use encoder/transformer/decoder combo

for discriminator, use patchGAN

# making generator

we gotta make residual blocks

In [ ]:
class CycleGenerator(nn.module):
    def __init__(self):
        super(CycleGenerator, self).__init__()
        # do we need this many filter channels 
        # if we're doing a 1 channel image rather than 3 channel?
        
        #encoder section
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64,
                               kernel_size=(7, 7), padding=(1, 1))
        self.conv2 = nn.Conv2d(64, 128, (3, 3), padding=(1, 1), stride=2)
        self.conv3 = nn.Conv2d(128, 256, (3, 3), padding=(1, 1), stride=2)
        
        # in the transformer
        self.conv4 = nn.Conv2d(1, 128, (3, 3), padding=(1, 1), stride=2)
        
        
        